In [ ]:
# insert data into hbase
import csv
import happybase
import time

batch_size = 1000
host = "hbase"
file_path = "IBM6.csv"
namespace = "sample_data"
row_count = 0
start_time = time.time()
table_name = b"rfic"


def connect_to_hbase():
    """ Connect to HBase server.
    This will use the host, namespace, table name, and batch size as defined in
    the global variables above.
    """
    _conn = happybase.Connection(host = host)
    _conn.open()
    tables = _conn.tables()
    print( "HBase has tables {0}".format(tables))

    if table_name not in tables:
      print( "Creating table {0}".format(table_name))
      _conn.create_table(table_name, { 'data': dict() } )


    table = _conn.table(table_name)
    batch = table.batch(batch_size = batch_size)
    return _conn, batch


def insert_row(batch, row):
    """ Insert a row into HBase.
    Write the row to the batch. When the batch size is reached, rows will be
    sent to the database.
    Rows have the following schema:
        [ id, keyword, subcategory, type, township, city, zip, council_district,
          opened, closed, status, origin, location ]
    """
    #print("Insert new row")
    batch.put(row[0] + '_' + row[1] , { "data:time": row[1], "data:open": row[2], "data:high": row[3],
        "data:low": row[4], "data:close": row[5], "data:volume": row[6] })


def read_csv():
    csvfile = open(file_path, "r", encoding='latin-1')
    csvreader = csv.reader(csvfile, delimiter=',', )
    return csvreader, csvfile

csvreader, csvfile = read_csv()
print ("Connected to file. name: %s" % (file_path))
# After everything has been defined, run the script.
print ("Connect to HBase. table name: %s, batch size: %i" % (table_name, batch_size))
conn, batch = connect_to_hbase()

# Loop through the rows. The first row contains column headers, so skip that
# row. Insert all remaining rows into the database.

for row in csvreader:
    row_count += 1
    if row_count == 1:
        pass
    else:
      try:
        us_date=row[0]
        mdy=us_date.split("/")
        row[0]=mdy[2] + '/' + mdy[0] + '/' + mdy[1]
        insert_row(batch, row)
      except:
        print(row_count)
        print(row)

print("Total rows: ", row_count)

# If there are any leftover rows in the batch, send them now.
batch.send()

# No matter what happens, close the file handle.

duration = time.time() - start_time
print(duration)

conn.close()

